# 数据预处理

举一个例子，我们首先(**创建一个人工数据集，并存储在CSV（逗号分隔值）文件**)
`../data/house_tiny.csv`中。
以其他格式存储的数据也可以通过类似的方式进行处理。
下面我们将数据集按行写入CSV文件中。


In [1]:
import os

os.makedirs(os.path.join('..', 'data'), exist_ok=True)
data_file = os.path.join('..', 'data', 'house_tiny.csv')
with open(data_file, 'w') as f:
    f.write('NumRooms,Alley,Price\n')  # 列名
    f.write('NA,Pave,127500\n')  # 每行表示一个数据样本
    f.write('2,NA,106000\n')
    f.write('4,NA,178100\n')
    f.write('NA,NA,140000\n')

要[**从创建的CSV文件中加载原始数据集**]，我们导入`pandas`包并调用`read_csv`函数。该数据集有四行三列。其中每行描述了房间数量（“NumRooms”）、巷子类型（“Alley”）和房屋价格（“Price”）。


In [4]:
!pip install pandas

  Using cached pandas-2.3.3-cp39-cp39-manylinux_2_24_x86_64.manylinux_2_28_x86_64.whl.metadata (91 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
Using cached pandas-2.3.3-cp39-cp39-manylinux_2_24_x86_64.manylinux_2_28_x86_64.whl (12.8 MB)
Using cached pytz-2025.2-py2.py3-none-any.whl (509 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [pandas]━━━━━━━━━━━ 2/3 [pandas]


In [5]:
# 如果没有安装pandas，只需取消对以下行的注释来安装pandas
# !pip install pandas
import pandas as pd

data = pd.read_csv(data_file)
print(data)

   NumRooms Alley   Price
0       NaN  Pave  127500
1       2.0   NaN  106000
2       4.0   NaN  178100
3       NaN   NaN  140000


# 处理缺失值

注意，“NaN”项代表缺失值。
[**为了处理缺失的数据，典型的方法包括*插值法*和*删除法*，**]
其中插值法用一个替代值弥补缺失值，而删除法则直接忽略缺失值。
在(**这里，我们将考虑插值法**)。

通过位置索引`iloc`，我们将`data`分成`inputs`和`outputs`，
其中前者为`data`的前两列，而后者为`data`的最后一列。
对于`inputs`中缺少的数值，我们用同一列的均值替换“NaN”项。

In [11]:
inputs, outputs = data.iloc[:, 0:2], data.iloc[:, 2]
#inputs = inputs.fillna(inputs.mean())
fill_values = {}

# 数值列：用均值
for col in inputs.select_dtypes(include='number').columns:
    fill_values[col] = inputs[col].mean()

# 字符串列：用众数或固定值
for col in inputs.select_dtypes(exclude='number').columns:
    if inputs[col].notna().any():
        fill_values[col] = inputs[col].mode()[0]
    else:
        fill_values[col] = 'None'

inputs = inputs.fillna(value=fill_values)
print(inputs)

   NumRooms Alley
0       3.0  Pave
1       2.0  Pave
2       4.0  Pave
3       3.0  Pave


[**对于`inputs`中的类别值或离散值，我们将“NaN”视为一个类别。**]
由于“巷子类型”（“Alley”）列只接受两种类型的类别值“Pave”和“NaN”，
`pandas`可以自动将此列转换为两列“Alley_Pave”和“Alley_nan”。
巷子类型为“Pave”的行会将“Alley_Pave”的值设置为1，“Alley_nan”的值设置为0。
缺少巷子类型的行会将“Alley_Pave”和“Alley_nan”分别设置为0和1。

In [12]:
inputs = pd.get_dummies(inputs, dummy_na=True)
print(inputs)

   NumRooms  Alley_Pave  Alley_nan
0       3.0        True      False
1       2.0        True      False
2       4.0        True      False
3       3.0        True      False


# 转换为张量格式

[**现在`inputs`和`outputs`中的所有条目都是数值类型，它们可以转换为张量格式。**]
当数据采用张量格式后，可以通过在 :numref:`sec_ndarray`中引入的那些张量函数来进一步操作。

In [13]:
import torch

X = torch.tensor(inputs.to_numpy(dtype=float))
y = torch.tensor(outputs.to_numpy(dtype=float))
X, y

(tensor([[3., 1., 0.],
         [2., 1., 0.],
         [4., 1., 0.],
         [3., 1., 0.]], dtype=torch.float64),
 tensor([127500., 106000., 178100., 140000.], dtype=torch.float64))

将包含 NaN 的类别变量（categorical variable）进行独热编码（One-Hot Encoding）时，把 NaN 本身当作一个有效类别来处理的做法。这在机器学习中是一种常见且合理的策略，尤其当缺失值本身具有信息量时（例如，“没有巷子”可能意味着某种房屋类型）。

正确做法：使用 pd.get_dummies 并保留 NaN 作为类别
默认情况下，pandas.get_dummies() 会忽略 NaN 值，即不会为 NaN 创建虚拟列。但你可以通过以下方式显式地将 NaN 视为一个类别：

方法：先将 NaN 转换为字符串 'NaN'（或 'None'），再做独热编码

In [14]:
import pandas as pd

# 原始数据
tests = pd.DataFrame({
    'NumRooms': [None, 2.0, 4.0, None],
    'Alley': ['Pave', None, None, None]
})

# 对 Alley 列：将 NaN 替换为字符串 'NaN'，使其成为一个显式类别
tests['Alley'] = tests['Alley'].fillna('NaN')

# 然后进行独热编码
alley_dummies = pd.get_dummies(tests['Alley'], prefix='Alley')

print(alley_dummies)

   Alley_NaN  Alley_Pave
0      False        True
1       True       False
2       True       False
3       True       False


In [16]:
# 1. 分离数值列和类别列
numeric_cols = tests.select_dtypes(include='number').columns
categorical_cols = tests.select_dtypes(exclude='number').columns

# 2. 数值列：用均值填充（或其他策略）
tests[numeric_cols] = tests[numeric_cols].fillna(tests[numeric_cols].mean())

# 3. 类别列：将 NaN 视为一个类别（转为字符串）
for col in categorical_cols:
    tests[col] = tests[col].astype(str).fillna('NaN')  # 注意：astype(str) 会把 None 变成 'None'，但我们显式 fillna 更安全

# 或更稳妥：
for col in categorical_cols:
    tests[col] = tests[col].fillna('NaN').astype(str)

# 4. 对所有类别列做独热编码
inputs_encoded = pd.get_dummies(tests, columns=categorical_cols, dummy_na=False)
# 注意：dummy_na=False 是默认值，因为我们已经把 NaN 转成了 'NaN' 字符串

print(inputs_encoded)

   NumRooms  Alley_NaN  Alley_Pave
0       3.0      False        True
1       2.0       True       False
2       4.0       True       False
3       3.0       True       False
